In [1]:
# MISC
import os
import sys

sys.path.append("..")
os.environ['MUJOCO_GL']='egl'
os.environ['CUDA_VISIBLE_DEVICES']='0'

# import shutup
# shutup.please()

import jax
import jax.numpy as jnp

import numpy as np
from functools import partial

# VIS
import matplotlib.pyplot as plt
import mediapy
import scienceplots
from rich.pretty import pprint
from tqdm.auto import tqdm

from utils.datasets import GCDataset
from agents.dynamics_aware_iql import GCIQLAgent
from hydra import initialize, compose
from omegaconf import OmegaConf

plt.style.use(['seaborn-v0_8-colorblind', 'seaborn-v0_8-notebook'])

GLOBAL_KEY = jax.random.key(42)

In [4]:
from gymnasium.envs.registration import register
from envs.mujoco.online_env_utils import make_online_env
from envs.mujoco.ant import AntEnv

register(
    id='myant',
    entry_point=AntEnv,
    max_episode_steps=256,
    reward_threshold=-3.75,
)

env = make_online_env("myant", default_ind=2)
env.reset()

/home/m_bobrin/miniconda3/envs/jax2/lib/python3.10/site-packages/gymnasium/envs/registration.py:644: UserWarning: WARN: Overriding environment myant already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


(array([ 7.04719524e-01,  9.99313711e-01, -2.70990918e-02,  7.11409957e-02,
         7.57582220e-03, -4.80141607e-02,  7.59342923e-04, -2.88579519e-03,
         1.72699560e-02, -3.88146414e-02, -7.37668173e-02,  1.04261126e-02,
        -7.05115764e-02, -3.81557898e-02,  1.11619924e-01, -6.87687054e-02,
        -1.40427754e-02, -3.01712324e-02, -1.50216908e-01,  6.19122246e-02,
        -1.81274235e-02, -4.92894953e-02,  1.11995381e-01,  2.70073691e-02,
        -1.19684335e-01,  3.58902491e-02, -1.51943498e-01,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.0